# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [32]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import matplotlib.pyplot as plt


# Import API key
from api_keys import geoapify_key

In [33]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
len(city_data_df)

551

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [ ]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lon",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map plot
map_plot_1

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [37]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_df = city_data_df[(city_data_df['Temp Max'] < 27) & (city_data_df['Temp Max'] > 21)\
                             & (city_data_df['Wind Speed'] < 4.5)\
                                & (city_data_df['Cloudiness'] == 0 )]
 
# # Drop any rows with null values
filtered_df1 = filtered_df.dropna()

# Display sample data
filtered_df1

,City_ID,City,Lat,Lon,Temp Max,Humidity,Cloudiness,Wind Speed,Country,Date
18,18,gat,31.6100,34.7642,26.38,100,0,0.28,IL,1690754538
60,60,tuktoyaktuk,69.4541,-133.0374,24.00,57,0,1.54,CA,1690754258
106,106,oil city,41.4339,-79.7064,24.67,60,0,2.06,US,1690754268
108,108,syracuse,37.0881,15.2792,24.65,72,0,0.43,IT,1690754560
203,203,fuyu,45.1833,124.8167,23.20,87,0,2.08,CN,1690754587
210,210,jardim,-21.4803,-56.1381,24.78,45,0,0.68,BR,1690754507
253,253,qiongshan,20.0058,110.3542,26.99,94,0,2.00,CN,1690754601
302,302,millinocket,45.6573,-68.7098,23.06,40,0,4.12,US,1690754614
450,450,anastacio,-20.4836,-55.8069,25.51,40,0,0.19,BR,1690754500
458,458,baiyin,36.5583,104.2081,21.59,66,0,1.73,CN,1690754656


### Step 3: Create a new DataFrame called `hotel_df`.

In [39]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_df1[['City','Country','Lat','Lon','Humidity']].copy()
hotel_df


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''


# Display sample data
hotel_df

,City,Country,Lat,Lon,Humidity,Hotel Name
18,gat,IL,31.6100,34.7642,100,
60,tuktoyaktuk,CA,69.4541,-133.0374,57,
106,oil city,US,41.4339,-79.7064,60,
108,syracuse,IT,37.0881,15.2792,72,
203,fuyu,CN,45.1833,124.8167,87,
210,jardim,BR,-21.4803,-56.1381,45,
253,qiongshan,CN,20.0058,110.3542,94,
302,millinocket,US,45.6573,-68.7098,40,
450,anastacio,BR,-20.4836,-55.8069,40,
458,baiyin,CN,36.5583,104.2081,66,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [41]:
# Set parameters to search for a hotel. FROM 6.3.Student Geoapify Drill class assignment 
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit":20,
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame for the "Pono Kai Resort Hotel in the City of Kappaa"
    latitude = row['Lat']
    longitude = row['Lon']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
        # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


        # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)


        # Convert the API response to JSON format
    name_address = name_address.json()
    
        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
            hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
            # If no hotel is found, set the hotel name as "No hotel found".
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"

        # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
gat - nearest hotel: No hotel found
tuktoyaktuk - nearest hotel: Hunter's B&B
oil city - nearest hotel: Days Inn by Wyndham Oil City Conference Center
syracuse - nearest hotel: Hotel Panorama
fuyu - nearest hotel: 松原天達名都大酒店
jardim - nearest hotel: Hotel Uchôa
qiongshan - nearest hotel: 全季酒店
millinocket - nearest hotel: Ruthie's Hotel Terrace
anastacio - nearest hotel: Hotel Araras
baiyin - nearest hotel: 红鹭宾馆
russell - nearest hotel: No hotel found


,City,Country,Lat,Lon,Humidity,Hotel Name
18,gat,IL,31.6100,34.7642,100,No hotel found
60,tuktoyaktuk,CA,69.4541,-133.0374,57,Hunter's B&B
106,oil city,US,41.4339,-79.7064,60,Days Inn by Wyndham Oil City Conference Center
108,syracuse,IT,37.0881,15.2792,72,Hotel Panorama
203,fuyu,CN,45.1833,124.8167,87,松原天達名都大酒店
210,jardim,BR,-21.4803,-56.1381,45,Hotel Uchôa
253,qiongshan,CN,20.0058,110.3542,94,全季酒店
302,millinocket,US,45.6573,-68.7098,40,Ruthie's Hotel Terrace
450,anastacio,BR,-20.4836,-55.8069,40,Hotel Araras
458,baiyin,CN,36.5583,104.2081,66,红鹭宾馆


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display
map_plot_2 = city_data_df.hvplot.points(
    "Lon",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map plot
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)